# Banelo Forecasting - ML Model Training

This notebook trains machine learning models for inventory forecasting.

## Steps:
1. Upload CSV data files
2. Explore and preprocess data
3. Engineer features
4. Train multiple models (Random Forest, XGBoost, LSTM)
5. Evaluate and compare performance
6. Export best model for Django integration

---

## 1. Setup & Install Dependencies

In [ ]:
# Install required packages
!pip install -q pandas numpy scikit-learn xgboost matplotlib seaborn joblib

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import xgboost as xgb
import joblib

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

## 2. Upload Data Files

Upload the CSV files exported from your Django app:

In [ ]:
# Option 1: Upload files directly
from google.colab import files
uploaded = files.upload()
print("\n✅ Files uploaded!")

In [ ]:
# Option 2: Mount Google Drive (if you stored files there)
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = '/content/drive/MyDrive/banelo_data/'

## 3. Load and Explore Data

In [ ]:
# Load data
print("📊 Loading data...")
sales_df = pd.read_csv('sales_data.csv')
products_df = pd.read_csv('products_data.csv')
daily_sales_df = pd.read_csv('daily_sales_aggregated.csv')

# Convert date columns
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'])
daily_sales_df['date'] = pd.to_datetime(daily_sales_df['date'])

print(f"\n✅ Data loaded successfully!")
print(f"   - Sales records: {len(sales_df):,}")
print(f"   - Products: {len(products_df):,}")
print(f"   - Daily aggregates: {len(daily_sales_df):,}")

In [ ]:
# Preview data
print("\n📋 Sales Data Preview:")
display(sales_df.head())

print("\n📦 Products Data Preview:")
display(products_df.head())

print("\n📈 Daily Sales Aggregated Preview:")
display(daily_sales_df.head())

In [ ]:
# Data statistics
print("\n📊 Data Statistics:")
print("\nSales by Category:")
print(sales_df.groupby('category')['quantity'].agg(['count', 'sum', 'mean']))

print("\nTop 10 Products by Sales Volume:")
top_products = sales_df.groupby('product_name')['quantity'].sum().sort_values(ascending=False).head(10)
print(top_products)

## 4. Data Visualization

In [ ]:
# Plot 1: Sales over time
plt.figure(figsize=(14, 6))
daily_total = sales_df.groupby(sales_df['order_date'].dt.date)['quantity'].sum()
plt.plot(daily_total.index, daily_total.values, marker='o', linewidth=2)
plt.title('Total Daily Sales Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Total Quantity')
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 2: Sales by category
plt.figure(figsize=(10, 6))
category_sales = sales_df.groupby('category')['quantity'].sum().sort_values(ascending=False)
sns.barplot(x=category_sales.values, y=category_sales.index, palette='viridis')
plt.title('Total Sales by Category', fontsize=16, fontweight='bold')
plt.xlabel('Total Quantity')
plt.ylabel('Category')
plt.tight_layout()
plt.show()

In [ ]:
# Plot 3: Top products
plt.figure(figsize=(12, 8))
top_20 = sales_df.groupby('product_name')['quantity'].sum().sort_values(ascending=False).head(20)
sns.barplot(x=top_20.values, y=top_20.index, palette='rocket')
plt.title('Top 20 Products by Sales Volume', fontsize=16, fontweight='bold')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Product')
plt.tight_layout()
plt.show()

## 5. Feature Engineering

In [ ]:
def engineer_features(daily_df):
    """
    Create features for ML model
    """
    print("🔧 Engineering features...")
    
    df = daily_df.copy()
    
    # Sort by date
    df = df.sort_values(['product_id', 'date'])
    
    # Time-based features
    df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek
    df['day_of_month'] = pd.to_datetime(df['date']).dt.day
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['week_of_year'] = pd.to_datetime(df['date']).dt.isocalendar().week
    
    # Rolling statistics (7-day window)
    df['rolling_mean_7d'] = df.groupby('product_id')['total_quantity'].transform(
        lambda x: x.rolling(window=7, min_periods=1).mean()
    )
    df['rolling_std_7d'] = df.groupby('product_id')['total_quantity'].transform(
        lambda x: x.rolling(window=7, min_periods=1).std().fillna(0)
    )
    df['rolling_max_7d'] = df.groupby('product_id')['total_quantity'].transform(
        lambda x: x.rolling(window=7, min_periods=1).max()
    )
    df['rolling_min_7d'] = df.groupby('product_id')['total_quantity'].transform(
        lambda x: x.rolling(window=7, min_periods=1).min()
    )
    
    # Rolling statistics (30-day window)
    df['rolling_mean_30d'] = df.groupby('product_id')['total_quantity'].transform(
        lambda x: x.rolling(window=30, min_periods=1).mean()
    )
    df['rolling_std_30d'] = df.groupby('product_id')['total_quantity'].transform(
        lambda x: x.rolling(window=30, min_periods=1).std().fillna(0)
    )
    
    # Lag features
    df['lag_1d'] = df.groupby('product_id')['total_quantity'].shift(1).fillna(0)
    df['lag_7d'] = df.groupby('product_id')['total_quantity'].shift(7).fillna(0)
    df['lag_14d'] = df.groupby('product_id')['total_quantity'].shift(14).fillna(0)
    
    # Trend features
    df['days_since_start'] = (pd.to_datetime(df['date']) - pd.to_datetime(df['date']).min()).dt.days
    
    # Category encoding
    le = LabelEncoder()
    df['category_encoded'] = le.fit_transform(df['category'])
    
    print(f"   ✅ Created {len(df.columns)} features")
    
    return df, le

# Apply feature engineering
featured_df, label_encoder = engineer_features(daily_sales_df)
print("\n✅ Feature engineering complete!")
print(f"\nFeature columns: {list(featured_df.columns)}")

## 6. Prepare Training Data

In [ ]:
# Define features and target
feature_columns = [
    'day_of_week', 'day_of_month', 'month', 'is_weekend', 'week_of_year',
    'rolling_mean_7d', 'rolling_std_7d', 'rolling_max_7d', 'rolling_min_7d',
    'rolling_mean_30d', 'rolling_std_30d',
    'lag_1d', 'lag_7d', 'lag_14d',
    'days_since_start', 'category_encoded',
    'num_transactions', 'avg_price'
]

target_column = 'total_quantity'

# Prepare X and y
X = featured_df[feature_columns].fillna(0)
y = featured_df[target_column]

print(f"📊 Training data shape: {X.shape}")
print(f"📊 Target data shape: {y.shape}")

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"\n✅ Train set: {X_train.shape[0]} samples")
print(f"✅ Test set: {X_test.shape[0]} samples")

## 7. Train Models

We'll train 3 different models and compare performance:

### Model 1: Linear Regression (Baseline)

In [ ]:
print("🤖 Training Linear Regression...")

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test)

# Metrics
lr_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
lr_mae = mean_absolute_error(y_test, y_pred_lr)
lr_r2 = r2_score(y_test, y_pred_lr)

print(f"\n✅ Linear Regression Results:")
print(f"   RMSE: {lr_rmse:.4f}")
print(f"   MAE: {lr_mae:.4f}")
print(f"   R² Score: {lr_r2:.4f}")

### Model 2: Random Forest (Recommended)

In [ ]:
print("🌲 Training Random Forest...")

rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

rf_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)

# Metrics
rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
rf_mae = mean_absolute_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)

print(f"\n✅ Random Forest Results:")
print(f"   RMSE: {rf_rmse:.4f}")
print(f"   MAE: {rf_mae:.4f}")
print(f"   R² Score: {rf_r2:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📊 Top 10 Important Features:")
print(feature_importance.head(10))

In [ ]:
# Plot feature importance
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=feature_importance.head(15), palette='viridis')
plt.title('Top 15 Feature Importances (Random Forest)', fontsize=16, fontweight='bold')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

### Model 3: XGBoost (High Performance)

In [ ]:
print("🚀 Training XGBoost...")

xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train, verbose=False)

# Predictions
y_pred_xgb = xgb_model.predict(X_test)

# Metrics
xgb_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
xgb_mae = mean_absolute_error(y_test, y_pred_xgb)
xgb_r2 = r2_score(y_test, y_pred_xgb)

print(f"\n✅ XGBoost Results:")
print(f"   RMSE: {xgb_rmse:.4f}")
print(f"   MAE: {xgb_mae:.4f}")
print(f"   R² Score: {xgb_r2:.4f}")

## 8. Model Comparison

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'XGBoost'],
    'RMSE': [lr_rmse, rf_rmse, xgb_rmse],
    'MAE': [lr_mae, rf_mae, xgb_mae],
    'R² Score': [lr_r2, rf_r2, xgb_r2]
})

print("\n📊 MODEL COMPARISON")
print("=" * 60)
display(comparison_df)

# Find best model
best_idx = comparison_df['R² Score'].idxmax()
best_model_name = comparison_df.loc[best_idx, 'Model']
print(f"\n🏆 Best Model: {best_model_name}")
print("=" * 60)

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# RMSE
axes[0].bar(comparison_df['Model'], comparison_df['RMSE'], color=['#3498db', '#2ecc71', '#e74c3c'])
axes[0].set_title('Root Mean Squared Error (Lower is Better)', fontweight='bold')
axes[0].set_ylabel('RMSE')
axes[0].tick_params(axis='x', rotation=45)

# MAE
axes[1].bar(comparison_df['Model'], comparison_df['MAE'], color=['#3498db', '#2ecc71', '#e74c3c'])
axes[1].set_title('Mean Absolute Error (Lower is Better)', fontweight='bold')
axes[1].set_ylabel('MAE')
axes[1].tick_params(axis='x', rotation=45)

# R² Score
axes[2].bar(comparison_df['Model'], comparison_df['R² Score'], color=['#3498db', '#2ecc71', '#e74c3c'])
axes[2].set_title('R² Score (Higher is Better)', fontweight='bold')
axes[2].set_ylabel('R² Score')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 9. Predictions Visualization

In [ ]:
# Plot actual vs predicted (Random Forest)
plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_pred_rf, alpha=0.5, s=30)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Values', fontsize=12)
plt.ylabel('Predicted Values', fontsize=12)
plt.title('Random Forest: Actual vs Predicted', fontsize=16, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Residual plot
residuals = y_test - y_pred_rf

plt.figure(figsize=(12, 6))
plt.scatter(y_pred_rf, residuals, alpha=0.5, s=30)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Values', fontsize=12)
plt.ylabel('Residuals', fontsize=12)
plt.title('Residual Plot (Random Forest)', fontsize=16, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Export Model for Django Integration

In [ ]:
# Choose the best model to export (Random Forest recommended)
final_model = rf_model
model_name = 'Random Forest'

# Create metadata dictionary
model_metadata = {
    'model_name': model_name,
    'model_type': 'RandomForestRegressor',
    'trained_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'feature_columns': feature_columns,
    'metrics': {
        'rmse': float(rf_rmse),
        'mae': float(rf_mae),
        'r2_score': float(rf_r2)
    },
    'feature_importance': feature_importance.to_dict('records')[:10]
}

# Package everything together
model_package = {
    'model': final_model,
    'metadata': model_metadata,
    'label_encoder': label_encoder,
    'feature_columns': feature_columns
}

# Save model
joblib.dump(model_package, 'forecasting_model.pkl', compress=3)
print("\n✅ Model saved successfully!")
print(f"\n📦 Model: forecasting_model.pkl")
print(f"   - Model Type: {model_name}")
print(f"   - Training Samples: {len(X_train):,}")
print(f"   - Test Samples: {len(X_test):,}")
print(f"   - RMSE: {rf_rmse:.4f}")
print(f"   - MAE: {rf_mae:.4f}")
print(f"   - R² Score: {rf_r2:.4f}")

print("\n📥 Download this file and place it in: ml_models/forecasting_model.pkl")
print("\n✅ Then run: python integrate_ml_model.py")

In [ ]:
# Download the model
from google.colab import files
files.download('forecasting_model.pkl')
print("\n✅ Model downloaded! Ready for Django integration.")

## 11. Generate Sample Predictions (Preview)

In [ ]:
# Create sample predictions for preview
sample_predictions = []

# Get unique products
unique_products = featured_df[['product_id', 'product_name', 'category']].drop_duplicates()

for _, product in unique_products.head(10).iterrows():
    # Get latest features for this product
    product_data = featured_df[featured_df['product_id'] == product['product_id']].tail(1)
    
    if len(product_data) > 0:
        X_sample = product_data[feature_columns].fillna(0)
        prediction = final_model.predict(X_sample)[0]
        
        sample_predictions.append({
            'product_name': product['product_name'],
            'category': product['category'],
            'predicted_daily_usage': round(prediction, 2),
            'current_avg': round(product_data['rolling_mean_7d'].values[0], 2)
        })

# Display predictions
predictions_df = pd.DataFrame(sample_predictions)
print("\n🔮 SAMPLE PREDICTIONS (Top 10 Products)")
print("=" * 70)
display(predictions_df)
print("\n✅ These predictions will be integrated into your Django dashboard!")

---

## ✅ Training Complete!

### Next Steps:

1. ✅ Download `forecasting_model.pkl`
2. ✅ Place it in your Django project: `ml_models/forecasting_model.pkl`
3. ✅ Run integration script: `python integrate_ml_model.py`
4. ✅ Sync to Firebase: `python sync_predictions_to_firebase.py`
5. ✅ View predictions in dashboard: `/dashboard/inventory/forecasting/`

---